In [25]:
%matplotlib inline
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression

In [2]:
def generate_data_matrix(spatial_size, temporal_size, miss_ratio=0):
    data_matrix = np.random.randint(1,5,size=(spatial_size,temporal_size))
    origin = data_matrix.copy()
    k = round(spatial_size*temporal_size*miss_ratio)
    ri = np.random.choice(spatial_size*temporal_size,k,replace=False)
    data_matrix.flat[ri] = 0
    return origin,data_matrix

In [150]:
origin,dm = generate_data_matrix(5,6,0.1)

In [272]:
# Applying K-nearest neighbors Estimation (AKE)

def ake(data_matrix):
    if data_matrix is not None:
        return_matrix = data_matrix.copy().astype(float)

        # zero_index = np.where(data_matrix.flatten()==0)[0]
        # rec_array = np.zeros_like(zero_index,dtype=float)

        spatial_length, temporal_length = data_matrix.shape

        zero_index = np.where(data_matrix.flatten() == 0)[0]
        rec_array_global = np.zeros_like(zero_index,dtype=float)

        temporal_index = zero_index % temporal_length
        spatial_index = zero_index / temporal_length
        spatial_index = spatial_index.astype(int)

        # print(zero_index)
        # print('ti',temporal_index)
        # print('si',spatial_index)
        index = np.arange(temporal_length)
        for i in range(len(zero_index)):
            # print(zero_index[i])
            sim_array = np.ones(spatial_length)
            # sim_array[spatial_index[i]] = 0
            tmp = data_matrix.T[temporal_index[i]]
            tmp_zero_index = np.where(tmp == 0)[0]
            sim_array[tmp_zero_index] = 0
            sim_array_index = np.where(sim_array==1)[0]
            rec_array = np.zeros_like(sim_array)
            # print(rec_array.shape)
            # tmp_one_index = np.where(tmp != 0)[0]
            for ii in sim_array_index:
                sim_array[ii],tmp_rec_array = sim_temporal(data_matrix,spatial_index[i],ii)
                if tmp_rec_array is not None:
                    rec_array[ii] = tmp_rec_array[temporal_index[i]]
                    # print(tmp_rec_array.shape)
            # print(sim_array)
            sim_array = sim_array / np.sum(sim_array)
            # print(sim_array)
            # print(rec_array)
            # print('---')
            rec_array_global[i] = np.dot(sim_array,rec_array)
            if np.isnan(rec_array_global[i]) or np.isinf(rec_array_global[i]):
                rec_array_global[i] = 0
            

        # print('rec',rec_array_global)
        return_matrix.flat[zero_index] = rec_array_global
        # print(rec_array.shape)


            # print(sim_array)
        # for i in range(temporal_length):
        #     tmp = data_matrix.T[i]
        #     print(tmp)

        #     zero_index = np.where(tmp == 0)[0]
        #     true_index = np.where(tmp != 0)[0]
        #     rec_array = np.zeros_like(zero_index,dtype=float)
        #     for j,mis in enumerate(zero_index):
        #         dis_array = 

        

        return return_matrix
    else:
        return None

# origin,dm = generate_data_matrix(5,5,0.2)
ake(dm)-dm



array([[0.        , 0.        , 3.37142857, 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 5.31023218, 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ]])

In [266]:
dm

array([[1, 3, 0, 2, 4],
       [2, 4, 2, 3, 3],
       [4, 2, 1, 0, 2],
       [3, 3, 4, 3, 4],
       [0, 2, 0, 3, 0]])

In [226]:
def sim_temporal(sub_data,source_index,target_index):
    # source_index，指数据较多项
    # target_index，指数据缺失项
    source_non_zero_index = np.where(sub_data[source_index] != 0)[0]
    target_non_zero_index = np.where(sub_data[target_index] != 0)[0]

    intersect = np.intersect1d(source_non_zero_index,target_non_zero_index)
    # 取消过短的序列
    if len(intersect) < 3:        
        return 0,None

    diff1 = sub_data[source_index][intersect]
    diff2 = sub_data[target_index][intersect]
    # print(diff1)
    # print(diff2)
    reg = LinearRegression().fit(diff1.reshape(-1,1),diff2.reshape(-1,1))

    # print(reg.predict(np.array([3]).reshape(-1,1)))
    # print('cof',reg.coef_)
    # print('inter',reg.intercept_)
    
    target_mean = np.mean(diff2)
    # print('tm',target_mean)
    target_square = np.square(diff2 - target_mean)
    # print('sq',target_square)
    # print(target_mean)

    rec = reg.predict(diff1.reshape(-1,1))
    rec = rec.reshape(1,-1)[0]
    # print('rec',rec)
    rec_square = np.square(rec-target_mean)
    
    Rj = np.sum(rec_square) / np.sum(target_square)
    if np.isnan(Rj):
        Rj = 0
    elif np.isinf(Rj):
        Rj = 999

    # print('Rj',Rj)

    # print('rec_square',rec_square)
    # print('ts',target_square)
    
    full = sub_data[source_index]
    # print(tmp)
    # print('coef',reg.coef_)
    # print('inter',reg.intercept_)
    # print(full)
    rec_full = reg.predict(full.reshape(-1,1)).reshape(1,-1)[0]
    # print('rec_full',rec_full)
    # # print(diff1.reshape(-1,1))
    # # print(np.array([diff1]))
    # print(diff1.reshape(1,-1))
    # print(diff2)
    return Rj,rec_full

sim_temporal(test,1,0)

    # if np.isinf(sim):
    #     sim = 999
    # if np.isnan(sim):
    #     sim = 0
    # return sim    

(0.9696969696969698,
 array([ 5.54545455,  4.09090909, -0.27272727,  2.63636364,  1.18181818,
         4.09090909]))

In [151]:
print(dm)
# sim_temporal(dm,0,2)

[[0 4 1 3 1 4]
 [4 3 0 2 1 3]
 [2 1 2 0 4 4]
 [3 2 2 3 3 2]
 [4 4 4 1 4 4]]


In [166]:
test

array([[0, 4, 1, 3, 1, 4],
       [4, 3, 0, 2, 1, 3],
       [2, 1, 2, 0, 4, 4],
       [3, 2, 2, 3, 3, 2],
       [4, 2, 4, 2, 8, 8]])

In [165]:
test[4] = np.array([4,2,4,2,8,8])